In [1]:

import sqlite3
import pandas as pd

# get the data from the database
def extract_data(sqlite_file, model_name, visualize=False):
    conn = sqlite3.connect(sqlite_file)
    nvtx_df = pd.read_sql_query("SELECT start, end, textId FROM NVTX_EVENTS", conn)
    stringID_df = pd.read_sql_query("SELECT * FROM StringIds", conn)
    conn.close()
    
    nvtx_df['duration(ms)'] = ((nvtx_df['end'] - nvtx_df['start']) / 1000000) # (ns -> ms)
    
    nvtx_df = pd.merge(nvtx_df, stringID_df, left_on='textId', right_on='id', how='left')
    nvtx_df = nvtx_df.drop(['textId', 'id', 'start', 'end'], axis=1)
    nvtx_df = nvtx_df.rename(columns={'value': 'function_name'})
    
    functionList = ["3D_backbone", "bbox_head", "find_centers","post_processing", "reader", "transformer"]
    # get desired function list
    nvtx_df = nvtx_df[nvtx_df['function_name'].isin(functionList)]
    nvtx_df = nvtx_df.groupby(['function_name']).mean()
    
    if visualize:
        title = model_name + ' profiling result(%)'
        nvtx_df.plot.pie(y='duration(ms)', figsize=(12, 5), title=title, autopct='%1.1f%%', xlabel='', ylabel='', legend=False)
    
    # total duration을 구함
    total_duration = nvtx_df['duration(ms)'].sum()
    # nvtx_df.loc['total_duration'] = total_duration
    # some pandas+numpy version need this line
    nvtx_df.loc['total_duration'] = [total_duration] * len(nvtx_df.columns)
    nvtx_df['model_name'] = model_name
    
    return nvtx_df

# get the data from the database
def extract_data_wofirst(sqlite_file, model_name, visualize=False):
    conn = sqlite3.connect(sqlite_file)
    nvtx_df = pd.read_sql_query("SELECT start, end, textId FROM NVTX_EVENTS", conn)
    stringID_df = pd.read_sql_query("SELECT * FROM StringIds", conn)
    conn.close()
    
    nvtx_df['duration(ms)'] = ((nvtx_df['end'] - nvtx_df['start']) / 1000000) # (ns -> ms)
    
    nvtx_df = pd.merge(nvtx_df, stringID_df, left_on='textId', right_on='id', how='left')
    nvtx_df = nvtx_df.drop(['textId', 'id', 'start', 'end'], axis=1)
    nvtx_df = nvtx_df.rename(columns={'value': 'function_name'})
    
    functionList = ["3D_backbone", "bbox_head", "find_centers","post_processing", "reader", "transformer"]
    # get desired function list
    nvtx_df = nvtx_df[nvtx_df['function_name'].isin(functionList)]
    # remove fisrt elements. suppose there exist all column
    nvtx_df = nvtx_df[len(functionList):].groupby(['function_name']).mean()
    
    if visualize:
        title = model_name + ' profiling result(%)'
        nvtx_df.plot.pie(y='duration(ms)', figsize=(12, 5), title=title, autopct='%1.1f%%', xlabel='', ylabel='', legend=False)
    
    # total duration을 구함
    total_duration = nvtx_df['duration(ms)'].sum()
    # nvtx_df.loc['total_duration'] = total_duration
    # some pandas+numpy version need this line
    nvtx_df.loc['total_duration'] = [total_duration] * len(nvtx_df.columns)
    nvtx_df['model_name'] = model_name
    
    return nvtx_df


In [4]:
result_dir = 'results/time'
# file_names = ['baseline', 'poolformer', 'mem_opt', 'trt_32', 'trt_16', 'backbone_opt']
file_names = ['nusc_baseline', 'nusc_poolformer', 'waymo_baseline', 'waymo_poolformer']

dataframes = []
for file_name in file_names:
    sqlite_file = f'{result_dir}/{file_name}.sqlite'
    model_df = extract_data(sqlite_file, file_name, visualize=False)
    dataframes.append(model_df)

combined_df = pd.concat(dataframes)
combined_df = combined_df.reset_index()
combined_df = combined_df.pivot(index='model_name', columns='function_name', values='duration(ms)')

# column reordering
column_order = ['reader', '3D_backbone', 'find_centers', 'transformer', 'bbox_head', 'post_processing', 'total_duration']
combined_df = combined_df.reindex(column_order, axis=1)

# row_order = ['baseline', 'poolformer', 'mem_opt', 'trt_32', 'trt_16', 'backbone_opt']
combined_df = combined_df.reindex(file_names)

In [3]:
combined_df

function_name,reader,3D_backbone,find_centers,transformer,bbox_head,post_processing,total_duration
model_name,,,,,,,
nusc_baseline,33.748474,255.297712,542.523685,215.200668,32.360289,107.565390,1186.696219
nusc_poolformer,33.258818,252.126935,541.193039,4.779479,31.238679,60.625330,923.222279
waymo_baseline,23.155461,539.684724,822.780699,227.832755,5.669155,159.804685,1778.927479
waymo_poolformer,22.003301,541.124524,661.480508,4.208475,4.773646,108.392458,1341.982913


In [5]:
combined_df

function_name,reader,3D_backbone,find_centers,transformer,bbox_head,post_processing,total_duration
model_name,,,,,,,
nusc_baseline,36.327270,283.049713,908.114586,241.350625,45.296685,109.002331,1623.141210
nusc_poolformer,34.633597,278.036380,905.803374,5.445687,33.807598,62.433748,1320.160385
waymo_baseline,27.497785,676.252084,1514.826639,364.813223,29.229132,158.140737,2770.759601
waymo_poolformer,25.716798,675.778415,1317.844098,6.883052,10.643997,107.493137,2144.359497
